In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os


2024-02-25 03:12:13.027696: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 03:12:13.027851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 03:12:13.196693: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
#import the mobilenetv2 without the last layer
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)

9406464/9406464 [==============================] - 0s 0us/step


In [5]:
#add other layers
x = tf.keras.layers.GlobalAvgPool2D()(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
#the output layer for binary classification
x = tf.keras.layers.Dense(4,activation="softmax")(x)

model = tf.keras.Model(inputs = base_model.input , outputs = x)

In [6]:
#freezing the original model's layers
for layer in base_model.layers:
    layer.trainable = False

In [7]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [11]:
# define the directory containing training images
directory = '/kaggle/input/abdous-egg-experiment/dataset/dataset/day_'
target_size = (224, 224)

# initialize lists to store images and labels
images = []
labels = []

# iterate over each file in the directory
for i in range(4):
    for filename in os.listdir(directory+str(i+1)):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
            # load and preprocess the image
            img = tf.keras.preprocessing.image.load_img(os.path.join(directory+str(i+1), filename), target_size=target_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
            # add the preprocessed image to the list of images
            images.append(img_array)
            labels.append(i)

# convert lists to numpy arrays
X_train = np.array(images)
Y_train = np.array(labels)

In [15]:
Y_train.shape

(3798,)

In [ ]:
model.fit(X_train,Y_train,epochs = 10)

Epoch 1/10
119/119 [==============================] - 92s 776ms/step - loss: 0.1483 - accuracy: 0.9463
Epoch 2/10
119/119 [==============================] - 93s 782ms/step - loss: 0.1416 - accuracy: 0.9479
Epoch 3/10
119/119 [==============================] - 93s 785ms/step - loss: 0.1296 - accuracy: 0.9518
Epoch 4/10
119/119 [==============================] - 94s 794ms/step - loss: 0.1287 - accuracy: 0.9476
Epoch 5/10
119/119 [==============================] - 93s 785ms/step - loss: 0.1237 - accuracy: 0.9542
Epoch 6/10
 77/119 [==================>...........] - ETA: 33s - loss: 0.1168 - accuracy: 0.9554

In [17]:
# define the directory containing training images
directory = '/kaggle/input/abdous-egg-experiment/dataset/test'
target_size = (224, 224)

# initialize lists to store images and labels
images = []
names = []
files = os.listdir(directory)
files.sort()
for filename in files:
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join(directory, filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        images.append(img_array)
        names.append(filename)
# convert lists to numpy arrays
X_test = np.array(images)


In [18]:
predictions = model.predict(X_test)

14/14 [==============================] - 11s 729ms/step


In [19]:
predictionsf = np.argmax(predictions,axis=1)+1
predictionsf

array([3, 1, 4, 4, 2, 3, 1, 3, 3, 4, 3, 1, 2, 3, 1, 4, 2, 3, 2, 3, 4, 1,
       3, 1, 4, 4, 2, 4, 1, 2, 3, 2, 3, 4, 3, 3, 1, 3, 3, 4, 1, 2, 2, 3,
       3, 3, 4, 2, 2, 1, 3, 3, 4, 1, 3, 2, 1, 3, 3, 4, 3, 3, 2, 3, 1, 1,
       4, 4, 3, 1, 3, 2, 3, 1, 4, 3, 4, 1, 3, 1, 1, 4, 1, 3, 1, 3, 2, 4,
       2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 1, 4, 2, 4, 2, 4, 3, 4, 1, 1, 2, 4,
       3, 2, 2, 2, 4, 2, 1, 2, 2, 2, 2, 3, 4, 4, 1, 4, 2, 2, 3, 1, 4, 2,
       3, 2, 2, 4, 1, 3, 3, 2, 4, 1, 2, 2, 3, 1, 4, 4, 2, 4, 1, 2, 4, 3,
       1, 3, 3, 4, 1, 4, 2, 2, 1, 1, 1, 2, 3, 1, 4, 2, 1, 2, 3, 3, 3, 1,
       3, 1, 1, 3, 1, 4, 3, 3, 4, 4, 4, 3, 2, 4, 3, 4, 2, 1, 2, 3, 2, 2,
       2, 2, 2, 3, 4, 2, 4, 2, 4, 4, 1, 4, 4, 1, 3, 2, 2, 2, 3, 1, 1, 3,
       2, 4, 2, 4, 3, 2, 2, 1, 1, 1, 4, 2, 3, 3, 2, 2, 4, 4, 1, 2, 3, 4,
       4, 1, 1, 2, 3, 2, 4, 2, 4, 1, 4, 2, 1, 2, 1, 1, 2, 3, 2, 4, 4, 3,
       3, 3, 2, 2, 2, 2, 2, 4, 4, 3, 4, 1, 1, 2, 4, 1, 1, 4, 4, 1, 1, 2,
       3, 3, 3, 1, 2, 3, 1, 1, 4, 4, 3, 4, 3, 1, 2,

In [22]:
predictions_df = pd.DataFrame({"ID" : np.array(names),"Target": predictionsf})

In [24]:
predictions_df.to_csv("pb3.csv" , index = False)